In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TRANSFORMERS_CACHE'] = '/home/yunli/generative_agent/huggingface'

from langchain.llms import HuggingFacePipeline
import networkx as nx
from transformers import pipeline
import re
from creat import creat_world
from prompt import *
from server import logger
from datetime import datetime
import torch


logger.configure(dir = './log/log-' + str(datetime.now()))
logger.log("creating data loader...")

Logging to ./log/log-2023-12-12 11:46:26.079084
creating data loader...


In [2]:
pipeline = pipeline(model="meta-llama/Llama-2-7b-chat-hf", device_map="auto", torch_dtype=torch.float16)
model = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
locations, agents = creat_world(model)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, sup

In [4]:
action_results = {}
back_know = []
for i in town_people.keys(): # generate bk knowledge for everyone
  back_know.append(i+': '+ agents[i].get_summary(force_refresh=False))
# bk = [x for x in back_know]
for i in town_people.keys(): # add to people mem
  agents[i].memory.add_memory(' You know the following about people: ' + '. '.join(back_know))
  action_results[i] = i + 'is' + town_people[i]["status"]

In [5]:
global_time = 0
for repeats in range(4):
  global_time += 1
  logger.log("In global time " + str(global_time))
  action_prompts = {}
  people_description = []
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
        people_description.append(action_results[i])

    # for name in people:
      # prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
    # for i in people: 
    #   if i not in action_results: # initialize action results as the status, and add agent mem with description
    #     action_results[i] = agents[i].get_summary(force_refresh=False)
    for i in people: # add observation to memory and react
      logger.log("Mind Tree of people: ", i)

      others = [x for x in people if x != i]
      observation = "You are {}.You are currently in {} with the following description: {}. \
      It is currently {}:00. The following people are in this area: {}. You can interact with them.". \
      format(i, location, town_areas[location], str(global_time), ', '.join(others))

      others_des = [x for x in people_description if i+': ' not in x]
      observation += ' You know the following about people: ' + '. '.join(others_des)

      logger.log("For people %s, The observation is: %s \n" % (i, observation))

      agents[i].memory.add_memory(observation)
      _, reaction = agents[i].generate_reaction(observation)
      action_results[i] = reaction

      logger.log("action result is:  %s \n" %(reaction))

In global time1
Mind Tree of people:  Postowner Lius
For people Postowner Lius, The observation is: You are Postowner Lius.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 1:00. The following people are in this area: Retireadven Daran, Postmanager Linene, ToweMaster Harbin, Robber Terrill. You can interact with them. You know the following about people: Postowner Liusischatting with Post-Manager. Retireadven Daranisdriking the beer. Postmanager Lineneischatting with Postowner Lius. ToweMaster Harbiniscalculating the tax of the town. Robber Terrillischating with Redbrand ruffians members, secretly planing the robbery 

the entity i care most is: Postowner Lius 

the entities' action is: chatting with the Postmanager Linene in the observation. 

agent_summary_description is: Name: Postowner Lius (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Postowner Lius owns a trading post. 

relevant_mem

/home/yunli/anaconda3/envs/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yunli/anaconda3/envs/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yunli/anaconda3/envs/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


the entity i care most is: Retireadven Daran 



/home/yunli/anaconda3/envs/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


the entities' action is: drinking a beer in the town square. 



...

/home/yunli/anaconda3/envs/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yunli/anaconda3/envs/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yunli/anaconda3/envs/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


the entity i care most is: Robber Terrill 



/home/yunli/anaconda3/envs/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


the entities' action is: planning a robbery with Redbrand ruffians members in the town square of Phandalin. 



/home/yunli/anaconda3/envs/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
